### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [28]:
# Dependencies and Setup
import pandas as pd
import numpy as np


# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on="school_name")
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [29]:
total_schools=school_data_complete["school_name"].nunique()

total_students=school_data_complete["Student ID"].nunique()

school_budget_df=school_data_complete[["school_name","budget"]]
school_budget_df.head()
unique_rows=school_budget_df.drop_duplicates()
#http://www.datasciencemadesimple.com/get-unique-values-rows-dataframe-python-pandas/
unique_rows
total_budget=unique_rows["budget"].sum()
total_budget

average_math=school_data_complete["math_score"].mean()

average_reading=school_data_complete["reading_score"].mean()

passing_math_df=school_data_complete[school_data_complete["math_score"]>=70]
math_column=passing_math_df["math_score"]
math_count=math_column.count()
perc_math =(math_count/total_students)*100

passing_reading_df=school_data_complete[school_data_complete["reading_score"]>=70]
reading_column=passing_reading_df["reading_score"]
reading_count=reading_column.count()
perc_reading =(reading_count/total_students)*100



perc_overall_passing=(perc_math+perc_reading)/2

district_summary_df=pd.DataFrame({"Total Schools":[total_schools],
                                  "Total Students":[total_students],
                                  "Total Budget":[total_budget],
                                  "Average Math Score":[average_math],
                                  "Average Reading Score":[average_reading],
                                  "% Passing Math":[perc_math],
                                  "% Passing Reading":[perc_reading],
                                  "% Overall Passing":[perc_overall_passing]
                                 })


district_summary_df["Total Students"] = district_summary_df["Total Students"].map("{0:,}".format)
district_summary_df["Total Budget"] = district_summary_df["Total Budget"].map("${0:,.2f}".format)
district_summary_df["Average Math Score"] = district_summary_df["Average Math Score"].map("{0:,.2f}".format)
district_summary_df["Average Reading Score"] = district_summary_df["Average Reading Score"].map("{0:,.2f}".format)
district_summary_df["% Passing Math"] = district_summary_df["% Passing Math"].map("{0:,.2f}".format)
district_summary_df["% Passing Reading"] = district_summary_df["% Passing Reading"].map("{0:,.2f}".format)
district_summary_df["% Overall Passing"] = district_summary_df["% Overall Passing"].map("{0:,.2f}".format)

district_summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.99,81.88,74.98,85.81,80.39


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [30]:
print(school_data_complete.columns)

###########################################################################################################################
######################################SCHOOL NAME, TYPE, TOTAL NUMBER OF STUDENTS #########################################
###########################################################################################################################
#group by school name and type
grouped_name_type=school_data_complete.groupby(["school_name","type"],as_index=False)
#source:https://stackoverflow.com/questions/22407798/how-to-reset-a-dataframes-indexes-for-all-groups-in-one-step

grouped_name_type_df=pd.DataFrame(grouped_name_type.count())
grouped_name_type_df

#number of student ids is total number of students so rename student ID after groupby executed.
grouped_name_type_renamed=grouped_name_type_df.rename(columns={"Student ID":"Total Students",
                                                               "type":"School Type"})
grouped_name_type_renamed

#keep first 3 columns only i.e. school name, type, total students and create a dataframe with only these columns
table_0=grouped_name_type_renamed[["school_name","School Type","Total Students"]]
#print(group_1)

##########################################################################################################################
######################################### TOTAL SCHOOL BUDGET ############################################################
##########################################################################################################################
#calculate per total school budget
budget_0=school_data_complete[["school_name","budget"]]
budget_0_rename=budget_0.rename(columns={"budget":"Total School Budget"})
budget_1=budget_0_rename.drop_duplicates()
#print(budget_1)

#join two groups
table_1=pd.merge(table_0,budget_1, on="school_name")
table_1

###########################################################################################################################
############################################ PER STUDENT BUDGET ###########################################################
###########################################################################################################################
#Calculate per student bugdet
total_budget=table_1["Total School Budget"]
pd.to_numeric(total_budget)

total_students=table_1["Total Students"]
pd.to_numeric(total_students)

table_1["Per Student Budget"]=total_budget/total_students
table_1

##########################################################################################################################
########################################### AVERAGE MATH AND READING SCORES ##############################################
##########################################################################################################################

math_n_reading_df=pd.DataFrame(grouped_name_type["math_score","reading_score"].mean())
table_2=math_n_reading_df[["school_name","math_score"]]
table_2=math_n_reading_df.rename(columns={"math_score":"Average Math Score",
                                          "reading_score":"Average Reading Score"})
table_2

table_3=pd.merge(table_1,table_2,on="school_name")
del table_3["type"]
table_3

#########################################################################################################################
######################################### % PASSING MATH ###############################################################
########################################################################################################################
#percentage passing math
math_0=school_data_complete[school_data_complete["math_score"]>=70]
math_0
math_1=math_0.groupby(["school_name"],as_index=False)
#count number of students passing math
math_count=pd.DataFrame(math_1["Student ID"].count())
math_count_rename=math_count.rename(columns={"Student ID":"math_pass_count"})
math_count_rename
#merge on table 3, do calculation and drop count column

table_4=pd.merge(table_3,math_count_rename, on="school_name")
table_4

passing_math=table_4["math_pass_count"]
pd.to_numeric(passing_math)

total_students=table_4["Total Students"]
pd.to_numeric(total_students)

table_4["% Passing Math"]=(passing_math/total_students)*100
del table_4["math_pass_count"]
table_4
##########################################################################################################################
##################################### % PASSING READING ##################################################################
##########################################################################################################################
#percentage passing reading
reading_0=school_data_complete[school_data_complete["reading_score"]>=70]
reading_0
reading_1=reading_0.groupby(["school_name"],as_index=False)
#count number of students passing reading
reading_count=pd.DataFrame(reading_1["Student ID"].count())
reading_count_rename=reading_count.rename(columns={"Student ID":"reading_pass_count"})
reading_count_rename
#merge on table 4, do calculation and drop count column

table_5=pd.merge(table_4,reading_count_rename, on="school_name")
table_5

passing_reading=table_5["reading_pass_count"]
pd.to_numeric(passing_reading)

total_students=table_5["Total Students"]
pd.to_numeric(total_students)

table_5["% Passing Reading"]=(passing_reading/total_students)*100
del table_5["reading_pass_count"]
table_5

############################################################################################################################
############################################# % OVERALL PASSING ############################################################
############################################################################################################################
table_5["% Overall Passing"]=(table_5["% Passing Math"]+table_5["% Passing Reading"])/2
table_5

#new table to apply formatting to
table_6=table_5

#Formatting
table_6["Total School Budget"] = table_6["Total School Budget"].map("${0:,}".format)
table_6["Per Student Budget"] = table_6["Per Student Budget"].map("${0:.2f}".format)

########################## set school name to index and print final table  ############################################
table_7 = table_6.set_index("school_name")
table_7

Index(['Student ID', 'student_name', 'gender', 'grade', 'school_name',
       'reading_score', 'math_score', 'School ID', 'type', 'size', 'budget'],
      dtype='object')


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Bailey High School,District,4976,"$3,124,928",$628.00,77.048432,81.033963,66.680064,81.933280,74.306672
Cabrera High School,Charter,1858,"$1,081,356",$582.00,83.061895,83.975780,94.133477,97.039828,95.586652
Figueroa High School,District,2949,"$1,884,411",$639.00,76.711767,81.158020,65.988471,80.739234,73.363852
Ford High School,District,2739,"$1,763,916",$644.00,77.102592,80.746258,68.309602,79.299014,73.804308
Griffin High School,Charter,1468,"$917,500",$625.00,83.351499,83.816757,93.392371,97.138965,95.265668
Hernandez High School,District,4635,"$3,022,020",$652.00,77.289752,80.934412,66.752967,80.862999,73.807983
Holden High School,Charter,427,"$248,087",$581.00,83.803279,83.814988,92.505855,96.252927,94.379391
Huang High School,District,2917,"$1,910,635",$655.00,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,4761,"$3,094,650",$650.00,77.072464,80.966394,66.057551,81.222432,73.639992


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [31]:
top_five=table_7.sort_values("% Overall Passing", ascending=False)
top_five.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356",$582.00,83.061895,83.975780,94.133477,97.039828,95.586652
Thomas High School,Charter,1635,"$1,043,130",$638.00,83.418349,83.848930,93.272171,97.308869,95.290520
Pena High School,Charter,962,"$585,858",$609.00,83.839917,84.044699,94.594595,95.945946,95.270270
Griffin High School,Charter,1468,"$917,500",$625.00,83.351499,83.816757,93.392371,97.138965,95.265668
Wilson High School,Charter,2283,"$1,319,574",$578.00,83.274201,83.989488,93.867718,96.539641,95.203679


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [32]:
bottom_five=table_7.sort_values("% Overall Passing")
bottom_five.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363",$637.00,76.842711,80.744686,66.366592,80.220055,73.293323
Figueroa High School,District,2949,"$1,884,411",$639.00,76.711767,81.158020,65.988471,80.739234,73.363852
Huang High School,District,2917,"$1,910,635",$655.00,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,4761,"$3,094,650",$650.00,77.072464,80.966394,66.057551,81.222432,73.639992
Ford High School,District,2739,"$1,763,916",$644.00,77.102592,80.746258,68.309602,79.299014,73.804308


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [33]:
###############################################################################################################################
###################################### Create a pandas series for each grade ##################################################
###############################################################################################################################

school_data_complete.head()
_9th_grade=school_data_complete[(school_data_complete["grade"]=="9th")]
_10th_grade=school_data_complete[(school_data_complete["grade"]=="10th")]
_11th_grade=school_data_complete[(school_data_complete["grade"]=="11th")]
_12th_grade=school_data_complete[(school_data_complete["grade"]=="12th")]


##############################################################################################################################
##############################################  Group each series by school  #################################################
##############################################################################################################################
#                         9 TH  G R A D E 
_9th_grp=_9th_grade.groupby(["school_name"],as_index=False)
_9th_grp_df=pd.DataFrame(_9th_grp["math_score"].mean())
_9th_grp_rename=_9th_grp_df.rename(columns={"math_score":"9th"})
_9th_grp_rename

#                         10 TH  G R A D E 
_10th_grp=_10th_grade.groupby(["school_name"],as_index=False)
_10th_grp_df=pd.DataFrame(_10th_grp["math_score"].mean())
_10th_grp_rename=_10th_grp_df.rename(columns={"math_score":"10th"})
_10th_grp_rename

#                         11 TH  G R A D E 
_11th_grp=_11th_grade.groupby(["school_name"],as_index=False)
_11th_grp_df=pd.DataFrame(_11th_grp["math_score"].mean())
_11th_grp_rename=_11th_grp_df.rename(columns={"math_score":"11th"})
_11th_grp_rename

#                         12 TH  G R A D E 
_12th_grp=_12th_grade.groupby(["school_name"],as_index=False)
_12th_grp_df=pd.DataFrame(_12th_grp["math_score"].mean())
_12th_grp_rename=_12th_grp_df.rename(columns={"math_score":"12th"})
_12th_grp_rename


#Combine the series into a dataframe
_9th_n_10th=pd.merge(_9th_grp_rename,_10th_grp_rename, on="school_name")
_9_10_11=pd.merge(_9th_n_10th,_11th_grp_rename, on="school_name")
final_table=pd.merge(_9_10_11,_12th_grp_rename, on="school_name")
final_table_2=final_table.set_index("school_name")
final_table_2

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [34]:
###################################### Create a pandas series for each grade ##################################################
###############################################################################################################################

school_data_complete.head()
_9th_grade=school_data_complete[(school_data_complete["grade"]=="9th")]
_10th_grade=school_data_complete[(school_data_complete["grade"]=="10th")]
_11th_grade=school_data_complete[(school_data_complete["grade"]=="11th")]
_12th_grade=school_data_complete[(school_data_complete["grade"]=="12th")]


##############################################################################################################################
##############################################  Group each series by school  #################################################
##############################################################################################################################
#                         9 TH  G R A D E 
_9th_grp=_9th_grade.groupby(["school_name"],as_index=False)
_9th_grp_df=pd.DataFrame(_9th_grp["reading_score"].mean())
_9th_grp_rename=_9th_grp_df.rename(columns={"reading_score":"9th"})
_9th_grp_rename

#                         10 TH  G R A D E 
_10th_grp=_10th_grade.groupby(["school_name"],as_index=False)
_10th_grp_df=pd.DataFrame(_10th_grp["reading_score"].mean())
_10th_grp_rename=_10th_grp_df.rename(columns={"reading_score":"10th"})
_10th_grp_rename

#                         11 TH  G R A D E 
_11th_grp=_11th_grade.groupby(["school_name"],as_index=False)
_11th_grp_df=pd.DataFrame(_11th_grp["reading_score"].mean())
_11th_grp_rename=_11th_grp_df.rename(columns={"reading_score":"11th"})
_11th_grp_rename

#                         12 TH  G R A D E 
_12th_grp=_12th_grade.groupby(["school_name"],as_index=False)
_12th_grp_df=pd.DataFrame(_12th_grp["reading_score"].mean())
_12th_grp_rename=_12th_grp_df.rename(columns={"reading_score":"12th"})
_12th_grp_rename


#Combine the series into a dataframe
_9th_n_10th=pd.merge(_9th_grp_rename,_10th_grp_rename, on="school_name")
_9_10_11=pd.merge(_9th_n_10th,_11th_grp_rename, on="school_name")
final_table=pd.merge(_9_10_11,_12th_grp_rename, on="school_name")
final_table_2=final_table.set_index("school_name")
final_table_2

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [35]:
###############################################################################################################################
###################################Creating new data set with columns needed for analysis######################################
###############################################################################################################################
table_10=school_data_complete

table_11=table_6
table_12=pd.merge(table_10,table_11,on="school_name")
print(table_12.columns)

table_13=table_12[['school_name','Student ID','reading_score','math_score','size','budget','Total Students','Per Student Budget']]
table_13["Per Student Budget"] = table_13["Per Student Budget"].str.replace('$', '')
table_13["Per Student Budget"] = table_13["Per Student Budget"].astype(float)

###############################################################################################################################
############################################### Binning Process ################################################################
bins = [0, 585, 615, 645, 675]
# Create the names for the five bins
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]
table_13["Spending Ranges (Per Student)"] = pd.cut(table_13["Per Student Budget"], bins,labels=group_names,include_lowest=True,right=False)

###############################################################################################################################
########################################### Groupings by Spending ranges #######################################################

table_14=table_13.groupby(["Spending Ranges (Per Student)"],as_index=False)

#                                   ### average math and reading ###
math_reading_avg=pd.DataFrame(table_14["math_score","reading_score"].mean())

math_reading_avg = math_reading_avg.rename(columns={"math_score":"Average Math Scores",
                                                    "reading_score":"Average Reading Scores"})


#                                   ###counting number of rows per group ###
rows_per_group_m=pd.DataFrame(table_14["math_score"].count())
rows_per_group_m =rows_per_group_m.rename(columns={"math_score":"number per group m"})
rows_per_group_r=pd.DataFrame(table_14["reading_score"].count())
rows_per_group_r =rows_per_group_r.rename(columns={"reading_score":"number per group r"})


#                                   ### % Passing Math ###
math_data=table_13[table_13["math_score"]>=70]
pass_math_grp=math_data.groupby(["Spending Ranges (Per Student)"],as_index=False)
pass_math=pd.DataFrame(pass_math_grp["math_score"].count()) 
pass_math=pass_math.rename(columns={"math_score":"num_passing_math"})
df_merge_math=pd.merge(pass_math,rows_per_group_m,on="Spending Ranges (Per Student)")
df_merge_math["% Passing Math"] = (df_merge_math["num_passing_math"]/df_merge_math["number per group m"])*100
passing_math_df = df_merge_math[["Spending Ranges (Per Student)","% Passing Math"]]


#                                   ### merge tables ###
intermediate_table=pd.merge(math_reading_avg,passing_math_df,on="Spending Ranges (Per Student)")


#                                   ### % Passing Reading ###
reading_data=table_13[table_13["reading_score"]>=70]
pass_read_grp=reading_data.groupby(["Spending Ranges (Per Student)"],as_index=False)
pass_read=pd.DataFrame(pass_read_grp["reading_score"].count())
pass_read=pass_read.rename(columns={"reading_score":"num_passing_read"})
df_merge_read=pd.merge(pass_read,rows_per_group_r,on="Spending Ranges (Per Student)")
df_merge_read["% Passing Reading"]=(df_merge_read["num_passing_read"]/df_merge_read["number per group r"])*100
passing_reading_df=df_merge_read[["Spending Ranges (Per Student)","% Passing Reading"]]


#                                  ### % merge tables ###
final_table=pd.merge(intermediate_table,passing_reading_df,on="Spending Ranges (Per Student)")


#                                   ### % passing overall ###
final_table["% Overall Passing"]=(final_table["% Passing Math"]+final_table["% Passing Reading"])/2

# FINAL TABLE
final_table=final_table.set_index("Spending Ranges (Per Student)")
final_table


Index(['Student ID', 'student_name', 'gender', 'grade', 'school_name',
       'reading_score', 'math_score', 'School ID', 'type', 'size', 'budget',
       'School Type', 'Total Students', 'Total School Budget',
       'Per Student Budget', 'Average Math Score', 'Average Reading Score',
       '% Passing Math', '% Passing Reading', '% Overall Passing'],
      dtype='object')


C:\Users\clemi\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\clemi\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
C:\Users\clemi\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = val

,Average Math Scores,Average Reading Scores,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$585,83.363065,83.964039,93.702889,96.686558,95.194724
$585-615,83.529196,83.838414,94.124128,95.886889,95.005509
$615-645,78.061635,81.434088,71.400428,83.614770,77.507599
$645-675,77.049297,81.005604,66.230813,81.109397,73.670105


## Scores by School Size

* Perform the same operations as above, based on school size.

In [36]:
raw_data=school_data_complete
raw_data.columns
analysis_table=raw_data[['school_name','Student ID','reading_score','math_score','size']]


bins = [0, 1000, 2000, 5000]
# Create the names for the five bins
group_names = ["Small (<1000)","Medium (1000-2000)", "Large (2000-5000)"]

analysis_table["School Size"] = pd.cut(analysis_table["size"], bins,labels=group_names,include_lowest=True,right=False)



table_15=analysis_table.groupby(["School Size"],as_index=False)


#                                   ### average math and reading ###
math_reading_avg=pd.DataFrame(table_15["math_score","reading_score"].mean())

math_reading_avg = math_reading_avg.rename(columns={"math_score":"Average Math Scores",
                                                    "reading_score":"Average Reading Scores"})


#                                   ###counting number of rows per group ###
rows_per_group_m=pd.DataFrame(table_15["math_score"].count())
rows_per_group_m =rows_per_group_m.rename(columns={"math_score":"number per group m"})
rows_per_group_r=pd.DataFrame(table_15["reading_score"].count())
rows_per_group_r =rows_per_group_r.rename(columns={"reading_score":"number per group r"})


#                                   ### % Passing Math ###
math_data=analysis_table[analysis_table["math_score"]>=70]


pass_math_grp=math_data.groupby(["School Size"],as_index=False)
pass_math=pd.DataFrame(pass_math_grp["math_score"].count()) 
pass_math=pass_math.rename(columns={"math_score":"num_passing_math"})
df_merge_math=pd.merge(pass_math,rows_per_group_m,on="School Size")
df_merge_math["% Passing Math"] = (df_merge_math["num_passing_math"]/df_merge_math["number per group m"])*100
passing_math_df = df_merge_math[["School Size","% Passing Math"]]


#                                   ### merge tables ###
intermediate_table=pd.merge(math_reading_avg,passing_math_df,on="School Size")


#                                   ### % Passing Reading ###
reading_data=analysis_table[analysis_table["reading_score"]>=70]
pass_read_grp=reading_data.groupby(["School Size"],as_index=False)
pass_read=pd.DataFrame(pass_read_grp["reading_score"].count())
pass_read=pass_read.rename(columns={"reading_score":"num_passing_read"})
df_merge_read=pd.merge(pass_read,rows_per_group_r,on="School Size")
df_merge_read["% Passing Reading"]=(df_merge_read["num_passing_read"]/df_merge_read["number per group r"])*100
passing_reading_df=df_merge_read[["School Size","% Passing Reading"]]


#                                  ### % merge tables ###
final_table=pd.merge(intermediate_table,passing_reading_df,on="School Size")


#                                   ### % passing overall ###
final_table["% Overall Passing"]=(final_table["% Passing Math"]+final_table["% Passing Reading"])/2

# FINAL TABLE
final_table=final_table.set_index("School Size")
final_table


C:\Users\clemi\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,Average Math Scores,Average Reading Scores,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.828654,83.974082,93.952484,96.040317,94.996400
Medium (1000-2000),83.372682,83.867989,93.616522,96.773058,95.194790
Large (2000-5000),77.477597,81.198674,68.652380,82.125158,75.388769


## Scores by School Type

* Perform the same operations as above, based on school type

In [37]:
raw_data=school_data_complete
raw_data.columns
analysis_table=raw_data[['school_name','Student ID','reading_score','math_score','type']]


#                                     ### Groupings by type ###

grouping=analysis_table.groupby(["type"],as_index=False)


#                                   ### average math and reading ###
math_reading_avg=pd.DataFrame(grouping["math_score","reading_score"].mean())

math_reading_avg = math_reading_avg.rename(columns={"math_score":"Average Math Scores",
                                                    "reading_score":"Average Reading Scores"})


#                                   ###counting number of rows per group ###
rows_per_group_m=pd.DataFrame(grouping["math_score"].count())
rows_per_group_m =rows_per_group_m.rename(columns={"math_score":"number per group m"})
rows_per_group_r=pd.DataFrame(grouping["reading_score"].count())
rows_per_group_r =rows_per_group_r.rename(columns={"reading_score":"number per group r"})


#                                   ### % Passing Math ###
math_data=analysis_table[analysis_table["math_score"]>=70]


pass_math_grp=math_data.groupby(["type"],as_index=False)
pass_math=pd.DataFrame(pass_math_grp["math_score"].count()) 
pass_math=pass_math.rename(columns={"math_score":"num_passing_math"})
df_merge_math=pd.merge(pass_math,rows_per_group_m,on="type")
df_merge_math["% Passing Math"] = (df_merge_math["num_passing_math"]/df_merge_math["number per group m"])*100
passing_math_df = df_merge_math[["type","% Passing Math"]]


#                                   ### merge tables ###
intermediate_table=pd.merge(math_reading_avg,passing_math_df,on="type")


#                                   ### % Passing Reading ###
reading_data=analysis_table[analysis_table["reading_score"]>=70]
pass_read_grp=reading_data.groupby(["type"],as_index=False)
pass_read=pd.DataFrame(pass_read_grp["reading_score"].count())
pass_read=pass_read.rename(columns={"reading_score":"num_passing_read"})
df_merge_read=pd.merge(pass_read,rows_per_group_r,on="type")
df_merge_read["% Passing Reading"]=(df_merge_read["num_passing_read"]/df_merge_read["number per group r"])*100
passing_reading_df=df_merge_read[["type","% Passing Reading"]]


#                                  ### % merge tables ###
final_table=pd.merge(intermediate_table,passing_reading_df,on="type")


#                                   ### % passing overall ###
final_table["% Overall Passing"]=(final_table["% Passing Math"]+final_table["% Passing Reading"])/2
final_table=final_table.rename(columns={"type":"School Type"})

# FINAL TABLE
final_table=final_table.set_index("School Type")
final_table


,Average Math Scores,Average Reading Scores,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.406183,83.902821,93.701821,96.645891,95.173856
District,76.987026,80.962485,66.518387,80.905249,73.711818
